In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect


In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Create collection with a gerative model

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    # Configure Vectorizer - with AWS vectorizer and Titan model("amazon.titan-embed-text-v1")
    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        region="us-west-2",
    ),
    
    # Configure Generative model with AWS "cohere.command-r-plus-v1:0" model 
    generative_config=Configure.Generative.aws(
        region="us-west-2",
        model="anthropic.claude-3-sonnet-20240229-v1:0"
    ),

    properties=[  # Define properties (Optional)
        Property(name="chunk", data_type=DataType.TEXT),
        Property(name="arxiv_id", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="title", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="url", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="chunk_no", data_type=DataType.INT),
    ],
)

## Close the client when done

In [ ]:
client.close()